In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from lib.models.layer.score import ScoreAttention
import torch
x = torch.randn(1, 128, 768)
x_modal = torch.randn(1, 128, 768)
score = ScoreAttention(embed_dim=768)
mx = score(x, x_modal)
print(mx.shape)





torch.Size([1, 128, 768])
torch.Size([1, 128, 768])


In [2]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.plscore_RMT_sigma import PLScoreRMT
from lib.models.LeeNet.score_RMT_sigma_center import ScorePureRMTCENTER
from lib.models.layer.RMT import PatchMerging
from lib.models.head.center_predictor import CenterPredictor
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import build_dataloaders, get_optimizer_scheduler
from lib.config.cfg_loader import env_setting
from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch
import torch.backends.cudnn


def build_model(cfg):
    backbone = PLScoreRMT(down_sample=PatchMerging, cfg=cfg)

    stride = cfg.model.backbone.stride
    feat_sz = int(cfg.data.search.size / stride)
    head = CenterPredictor(inplanes=cfg.model.pureRMT.embed_dim[-1], channel=cfg.model.head.num_channels, feat_sz=feat_sz, stride=stride)
    model = ScorePureRMTCENTER(backbone, head, cfg)
    return model

/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.backends.cudnn.benchmark = True

cfg = env_setting(cfg_name="plscore_pureRMT_sigma_center.yaml")

loader_train, loader_val = build_dataloaders(cfg)

net = build_model(cfg)

focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)


trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

In [ ]:
trainer.train(cfg.train.epoch)